# Notebook de la dimensión Estado Actual - Proceso ETL

## 1. Configuración de conexiones (fuente y bodega)

In [1]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

# Leer configuración desde archivo externo
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# URLs de conexión
url_fuente = (f"{config_fuente['driver']}://{config_fuente['user']}:{config_fuente['password']}@"
              f"{config_fuente['host']}:{config_fuente['port']}/{config_fuente['db']}")

url_bodega = (f"{config_bodega['driver']}://{config_bodega['user']}:{config_bodega['password']}@"
              f"{config_bodega['host']}:{config_bodega['port']}/{config_bodega['db']}")

# Engines SQLAlchemy
engine_fuente = create_engine(url_fuente)
engine_bodega = create_engine(url_bodega)

## 2 y 3. Extracción de datos desde la base de datos - Extract and Transform

In [2]:
from sqlalchemy import inspect

# Saber las columnas que hay en la base de datos OLTP
inspector = inspect(engine_fuente)
tablas = inspector.get_table_names()
print("Tablas de la base de datos:")
print(tablas)

Tablas de la base de datos:
['admin_interface_theme', 'areas_cliente', 'auth_user_user_permissions', 'clientes_coordinador', 'clientes_datosmensajero', 'clientes_mensajeroaquitoy_clientes', 'auth_permission', 'auth_group', 'auth_user_groups', 'auth_user', 'authtoken_token', 'ciudad', 'cliente', 'clientes_mensajeroaquitoy', 'area', 'django_migrations', 'departamento', 'clientes_usuarioaquitoy', 'django_admin_log', 'django_session', 'mensajeria_novedadesservicio', 'mensajeria_tiponovedad', 'mensajeria_origenservicio', 'mensajeria_tipopago', 'mensajeria_estado', 'mensajeria_tiposervicio', 'mensajeria_servicio', 'mensajeria_estadosservicio', 'mensajeria_tipovehiculo', 'sede', 'auth_group_permissions', 'django_content_type', 'tipo_cliente', 'mensajeria_destinoservicio', 'mensajeria_documentoasociado']


In [3]:
# Consulta SQL para extraer los datos de la dimensión estado - aca ya se hizo el paso de transform desde la consulta SQL
query = """
SELECT 
    id AS id_estado,
    nombre AS nombre_estado,
    descripcion AS descripcion_estado
FROM mensajeria_estado;
"""

# Ejecutar la consulta y cargar los datos en un DataFrame
dim_estado_actual = pd.read_sql_query(query, engine_fuente)
dim_estado_actual

,id_estado,nombre_estado,descripcion_estado
0,4,Recogido por mensajero,Recogido por mensajero
1,5,Entregado en destino,Entregado en destino
2,3,Con novedad,Tiene novedad
3,6,Terminado completo,Terminado completo
4,1,Iniciado,Creado por el usuario
5,2,Con mensajero Asignado,Con mensajero Asignado


In [4]:
from datetime import datetime

# Agregar columna de fecha de carga
dim_estado_actual["fecha_carga"] = datetime.now().date()

## 4. Guardado de la dimensión - Load

In [5]:
# Guardar como archivo CSV
dim_estado_actual.to_csv("dim_estado_actual.csv", index=False)

In [6]:
# Cargar en la bodega de datos OLAP
dim_estado_actual.to_sql("dim_estado_actual", engine_bodega, if_exists="replace", index=False)

6

In [7]:
# Verificación del archivo
df_verificacion = pd.read_csv("dim_estado_actual.csv")
print(df_verificacion.head())

   id_estado           nombre_estado      descripcion_estado fecha_carga
0          4  Recogido por mensajero  Recogido por mensajero  2025-06-05
1          5    Entregado en destino    Entregado en destino  2025-06-05
2          3             Con novedad           Tiene novedad  2025-06-05
3          6      Terminado completo      Terminado completo  2025-06-05
4          1                Iniciado   Creado por el usuario  2025-06-05
